# All data

In [2]:
import numpy as np
import pandas as pd

In [41]:
ORIG_DIR = "original_data"
PROCESSED_DIR = "processed_data"

In [3]:
df = pd.read_csv(f'{ORIG_DIR}//all_2020-03-27_2021-04-15.csv')
df_state_new = pd.read_csv(f'{ORIG_DIR}//state_new_2020-03-27_2021-04-15.csv')
df_state_cumu = pd.read_csv(f'{ORIG_DIR}//state_cumu_2020-03-27_2021-04-15.csv')

In [5]:
df.head()

,Date,Recovered,Cumulative Recovered,Imported Case,Local Case,Active Case,New Case,Cumulative Case,ICU,Ventilator,Death,Cumulative Death,URL
0,27-03-20,44,259,NaN,NaN,NaN,130,2161,54,34,3,26,https://kpkesihatan.com/2020/03/27/kenyataan-a...
1,28-03-20,61,320,NaN,NaN,NaN,159,2320,73,54,1,27,https://kpkesihatan.com/2020/03/28/kenyataan-a...
2,29-03-20,68,388,NaN,NaN,NaN,150,2470,73,52,7,34,https://kpkesihatan.com/2020/03/29/kenyataan-a...
3,30-03-20,91,479,NaN,NaN,NaN,156,2626,94,62,3,37,https://kpkesihatan.com/2020/03/30/kenyataan-a...
4,31-03-20,58,537,NaN,NaN,NaN,140,2766,94,60,6,43,https://kpkesihatan.com/2020/03/31/kenyataan-a...


In [6]:
def preprocess_df(df):
    df.drop(columns='URL', inplace=True)    
    # must change the date format to datetime
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%y')
    df['Active Case'] = (df['Cumulative Case'] - df['Cumulative Recovered'] - df['Cumulative Death'])    
    df.set_index('Date', inplace=True)
    return df

In [7]:
df = preprocess_df(df)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 385 entries, 2020-03-27 to 2021-04-15
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Recovered             385 non-null    int64  
 1   Cumulative Recovered  385 non-null    int64  
 2   Imported Case         86 non-null     float64
 3   Local Case            86 non-null     float64
 4   Active Case           385 non-null    int64  
 5   New Case              385 non-null    int64  
 6   Cumulative Case       385 non-null    int64  
 7   ICU                   385 non-null    int64  
 8   Ventilator            385 non-null    int64  
 9   Death                 385 non-null    int64  
 10  Cumulative Death      385 non-null    int64  
dtypes: float64(2), int64(9)
memory usage: 36.1 KB


In [22]:
df_non_na = df.dropna()
df_non_na.head()

,Recovered,Cumulative Recovered,Imported Case,Local Case,Active Case,New Case,Cumulative Case,ICU,Ventilator,Death,Cumulative Death
Date,,,,,,,,,,,
2021-01-20,2374,127662,5.0,4003.0,41087,4008,169379,246,96,11,630
2021-01-21,2490,130152,8.0,3162.0,41755,3170,172549,260,103,12,642
2021-01-22,2554,132706,6.0,3625.0,42814,3631,176180,251,102,18,660
2021-01-23,4313,137019,11.0,4264.0,42769,4275,180455,260,103,7,667
2021-01-24,4427,141446,7.0,3339.0,41677,3346,183801,265,102,11,678


In [23]:
# Check whether all active cases have the correct numbers
print((df_non_na['Active Case'] == (df_non_na['Cumulative Case'] - df_non_na['Cumulative Recovered'] - df_non_na['Cumulative Death'])).all())
del df_non_na

True


In [25]:
# Check whether the daily cases and cumulative cases tally
test_df = df.copy()
test_df.loc['2020-03-27', 'New Case'] = test_df.loc['2020-03-27', 'Cumulative Case']
test_df.loc['2020-03-27', 'Recovered'] = test_df.loc['2020-03-27', 'Cumulative Recovered']
test_df.loc['2020-03-27', 'Death'] = test_df.loc['2020-03-27', 'Cumulative Death']
print((test_df['New Case'].cumsum() == test_df['Cumulative Case']).all())
print((test_df['Recovered'].cumsum() == test_df['Cumulative Recovered']).all())
print((test_df['Death'].cumsum() == test_df['Cumulative Death']).all())
del test_df

True
True
True


In [26]:
rolling_day = 7
df['SMA_new'] = df['New Case'].rolling(rolling_day, min_periods=1).mean().round(2)
df['SMA_new'].head()

Date
2020-03-27    130.00
2020-03-28    144.50
2020-03-29    146.33
2020-03-30    148.75
2020-03-31    147.00
Name: SMA_new, dtype: float64

In [59]:
df['SMA_death'] = df['Death'].rolling(rolling_day, min_periods=1).mean().round(2)
df['SMA_death'].head()

Date
2020-03-27    3.00
2020-03-28    2.00
2020-03-29    3.67
2020-03-30    3.50
2020-03-31    4.00
Name: SMA_death, dtype: float64

In [33]:
df['EMA_0.1'] = df['New Case'].ewm(alpha=0.1).mean().round(2)
df['EMA_0.3'] = df['New Case'].ewm(alpha=0.3).mean().round(2)
df['EMA_0.1'].head()

Date
2020-03-27    130.00
2020-03-28    145.26
2020-03-29    147.01
2020-03-30    149.62
2020-03-31    147.27
Name: EMA_0.1, dtype: float64

In [100]:
df.to_csv(f"{PROCESSED_DIR}//cleaned_all.csv")

# Monthly data

In [39]:
df_month = df.resample("M").sum()
df_month.reset_index(inplace=True)
df_month.Date = df_month.Date.apply(lambda dt: dt.replace(day=1))
df_month = df_month[['Date', 'Recovered', 'New Case', 'Death', 'ICU', 'Ventilator']]
df_month.head()

,Date,Recovered,New Case,Death,ICU,Ventilator
0,2020-03-01,322,735,20,388,262
1,2020-04-01,3634,3236,59,1881,1003
2,2020-05-01,2182,1817,13,480,194
3,2020-06-01,2001,820,6,124,85
4,2020-07-01,290,337,4,88,36


In [40]:
df_month.to_csv(f'{PROCESSED_DIR}//monthly_sum.csv', index=False)

# State Data

In [12]:
dfState = pd.read_csv(f'{ORIG_DIR}//state_new_2020-03-27_2021-04-15.csv')
dfStateCumu = pd.read_csv(f'{ORIG_DIR}//state_cumu_2020-03-27_2021-04-15.csv')

In [13]:
dfState.head()

,Date,JOHOR,JUMLAH KESELURUHAN,KEDAH,KELANTAN,MELAKA,NEGERI SEMBILAN,PAHANG,PERAK,PERLIS,PULAU PINANG,SABAH,SARAWAK,SELANGOR,TERENGGANU,WP KUALA LUMPUR,WP LABUAN,WP PUTRAJAYA
0,2020-03-27,20,130,1,4,0,6,4,8,0,6,10,15,36,2,16,0,2
1,2020-03-28,26,159,1,10,9,15,14,6,0,6,15,8,33,4,7,5,0
2,2020-03-29,24,150,1,15,1,9,12,11,0,1,0,11,33,2,28,0,2
3,2020-03-30,24,156,2,4,7,10,3,8,1,1,4,6,60,0,24,0,2
4,2020-03-31,16,140,0,4,2,9,3,5,1,6,5,21,32,0,34,0,2


In [14]:
def preprocess_state(df_state):
    # drop the unwanted column
    df_state.drop(columns='JUMLAH KESELURUHAN', inplace=True)
    # to remove digits surrounded by parenthesis, and also commas and space
    df_state = df_state.replace(r"(\(\d+\)|\,*\s*)", '', regex=True)
    # to change their dtypes to int to make sure they are valid
    df_state = df_state.set_index('Date').astype(int)
    
    return df_state

In [15]:
dfState = preprocess_state(dfState)
dfStateCumu = preprocess_state(dfStateCumu)

In [16]:
dfState.head()

,JOHOR,KEDAH,KELANTAN,MELAKA,NEGERI SEMBILAN,PAHANG,PERAK,PERLIS,PULAU PINANG,SABAH,SARAWAK,SELANGOR,TERENGGANU,WP KUALA LUMPUR,WP LABUAN,WP PUTRAJAYA
Date,,,,,,,,,,,,,,,,
2020-03-27,20,1,4,0,6,4,8,0,6,10,15,36,2,16,0,2
2020-03-28,26,1,10,9,15,14,6,0,6,15,8,33,4,7,5,0
2020-03-29,24,1,15,1,9,12,11,0,1,0,11,33,2,28,0,2
2020-03-30,24,2,4,7,10,3,8,1,1,4,6,60,0,24,0,2
2020-03-31,16,0,4,2,9,3,5,1,6,5,21,32,0,34,0,2


In [17]:
# problematic string at index 214
# dfStateCumu[(dfStateCumu.values == '28, 640')]

In [18]:
dfStateCumu.iloc[[214]]

,JOHOR,KEDAH,KELANTAN,MELAKA,NEGERI SEMBILAN,PAHANG,PERAK,PERLIS,PULAU PINANG,SABAH,SARAWAK,SELANGOR,TERENGGANU,WP KUALA LUMPUR,WP LABUAN,WP PUTRAJAYA
Date,,,,,,,,,,,,,,,,
2020-10-27,879,2052,179,310,1492,392,484,38,925,13155,819,4271,180,2977,346,141


In [19]:
# just to check whether still have problem
dfState.JOHOR.unique()

array([  20,   26,   24,   16,   19,   27,   21,    6,   30,   10,   18,
          7,   14,    8,   13,    9,    3,    2,    1,    4,    0,    5,
         15,   33,   60,   46,   37,   54,   68,  102,   78,   69,   66,
        125,   98,  108,  100,  105,  103,   45,   48,   67,   77,   75,
        156,  278,  167,  118,  115,  205,  412,  194,  191,  110,  607,
        112,  580,  179,  295,  428,  571, 1103,  298,  302,  321,  396,
        442,  535,  460,  719,  362,  329,  368,  470,  423,  466,  425,
        378,  529,  516, 1069,  717,  684,  792, 1020, 1068,  708,  964,
        664,  598,  585,  499,  490,  521,  369,  777,  742,  802,  492,
        269,  451,  436,  426,  730,  233,  431,  211,  318,  257,  276,
        314,  340,  163,  376,  255,  463,  139,  158,  120,  165,  273,
        170,  135,  101,  121,   91,  169,  142,  137,  130,  141,   86,
        161,  129,   81,  176,  171,   96,   83,   95,   55,   80,   61,
         74,   84,  182])

In [20]:
dfState.to_csv(f'{PROCESSED_DIR}//state_all.csv')
dfStateCumu.to_csv(f'{PROCESSED_DIR}//state_cumu.csv')

## Monthly

In [148]:
def preprocess_monthly_state(df_state):
    df_state = df.resample("M").sum()
    df_state.reset_index(inplace=True)
    df_state.Date = df_state.Date.apply(lambda dt: dt.replace(day=1))
    df_state.head()
    return df_state

In [140]:
dfState_month = preprocess_monthly_state(dfState_month)
dfState_month.head()

,Date,Recovered,Cumulative Recovered,Imported Case,Local Case,Active Case,New Case,Cumulative Case,ICU,Ventilator,Death,Cumulative Death,SMA_new,EMA_0.1,EMA_0.3,SMA_death
0,2020-03-01,322,1983,0.0,0.0,10193,735,12343,388,262,20,167,716.58,719.16,724.15,16.17
1,2020-04-01,3634,76354,0.0,0.0,66293,3236,145040,1881,1003,59,2393,3510.32,3880.68,3446.86,69.12
2,2020-05-01,2182,167455,0.0,0.0,43881,1817,214779,480,194,13,3443,1821.58,1931.42,1814.24,15.98
3,2020-06-01,2001,225307,0.0,0.0,23842,820,252725,124,85,6,3576,963.73,1244.49,945.47,6.02
4,2020-07-01,290,264528,0.0,0.0,3646,337,271966,88,36,4,3792,313.71,348.92,319.02,2.95


In [150]:
dfState_month.to_csv('processed_data//state_monthly.csv', index=False)